In [1]:
from measure import get_database_interface
import yaml

In [2]:
import pandas as pd
from ftplib import *
from io import StringIO
import io

In [3]:
from tqdm.auto import tqdm

In [4]:
def save_df_to_ftp(df, pretty_name, verbose=False):
    # function to put the generated values directly to FTP
    ftp = FTP('ftp-private.ebi.ac.uk')

    ftp.login('indigene_ftp', password)
    ftp.cwd('/upload/ct_scans/morphometric_analysis')
    
    buffer = StringIO()
    df.to_csv(buffer, header=True, index=False, columns=['fish_id', 'line_id', 'fish_sex', 'fish_age', 'fish_generation', 'measurement'])
    text = buffer.getvalue()
    bio = io.BytesIO(str.encode(text))
    
    ftp.storbinary(f'STOR {pretty_name}.txt', bio)
    
    if verbose:
        print(ftp.dir())

In [5]:
def list_ftp():
    # function to put the generated values directly to FTP
    ftp = FTP('ftp-private.ebi.ac.uk')

    ftp.login('indigene_ftp', password) # don't forget to remove the pass before the git-committing
    ftp.cwd('/upload/ct_scans/morphometric_analysis')
    
    print(ftp.dir())

In [6]:
def get_df(iface, par_name):
    # create dataframe from the database
        
    selection = iface.as_table([par_name, 'line', 'id', 'sex', 'age', 'generation'])
    df = pd.DataFrame(selection)
    df = df.loc[~df[par_name].isna()]
    df = df.explode(par_name).explode(par_name).explode(par_name)
    df = df.rename(columns={o:n for o,n in zip([par_name, 'line', 'id', 'sex', 'age', 'generation'], 
                                               ['measurement', 'line_id', 'fish_id', 'fish_sex', 'fish_age', 'fish_generation'])})
    return df

## Connect to the database and ftp

In [7]:
with open('measurement_configs/processing/medaka_mongo.yaml') as f:
    iface = get_database_interface(yaml.safe_load(f)['db'])

In [8]:
# Getting password for the FTP

import getpass
password = getpass.getpass(prompt='Password for the FTP, please: ')
try:
    list_ftp()
except error_perm:
    print('Password is incorrect! Please, re-run this cell and try again.')

Password for the FTP, please: ········
drwxrws---    2 ftp      ftp          4096 Apr 01  2021 backup
-rw-rw----    1 ftp      ftp          7509 Nov 09 14:23 brain.cerebellum.color_average.txt
-rw-rw----    1 ftp      ftp          7506 Nov 09 14:23 brain.cerebellum.color_std.txt
-rw-rw----    1 ftp      ftp          7274 Nov 09 14:23 brain.cerebellum.eccentricity_equatorial.txt
-rw-rw----    1 ftp      ftp          7212 Nov 09 14:23 brain.cerebellum.eccentricity_meridional.txt
-rw-rw----    1 ftp      ftp          5133 Nov 09 14:23 brain.cerebellum.surface_area.txt
-rw-rw----    1 ftp      ftp          5232 Nov 09 14:23 brain.cerebellum.volume.txt
-rw-rw----    1 ftp      ftp          5252 Nov 09 14:23 brain.epyphysis.color_average.txt
-rw-rw----    1 ftp      ftp          5247 Nov 09 14:23 brain.epyphysis.color_std.txt
-rw-rw----    1 ftp      ftp          4565 Nov 09 14:23 brain.epyphysis.surface_area.txt
-rw-rw----    1 ftp      ftp          4568 Nov 09 14:23 brain.epyphysis.volume.

## Adding meta-information to the database

In [9]:
scan_201811 = pd.read_excel('samplelist_complete.xlsx', sheet_name=4)
scan_201905 = pd.read_excel('samplelist_complete.xlsx', sheet_name=3)
scan_201912 = pd.read_excel('samplelist_complete.xlsx', sheet_name=2)
scan_202012 = pd.read_excel('samplelist_complete.xlsx', sheet_name=1)

all_scans = pd.concat([scan_201811, scan_201905, scan_201912, scan_202012])
all_scans = all_scans[all_scans['Scanned stained'].isin(['yes', 'Yes', 'Scanned', 'scanned'])]
all_scans = all_scans[['S.No', 'Line', 'Gen.', 'Sex', 'DOB', 'DOS']]
all_scans['age'] = (all_scans['DOS'] - pd.to_datetime(all_scans['DOB'], errors='coerce')).dt.days
all_scans = all_scans.drop(['DOS', 'DOB'], axis=1)

all_scans = all_scans.rename(columns={'S.No': 'id', 'Line': 'line', 'Gen.': 'generation', 'Sex': 'sex', 'age':'age'})
all_scans['id'] = pd.to_numeric(all_scans['id'], errors='coerce')
all_scans = all_scans[~all_scans.isna()['id']]
all_scans['id'] = all_scans['id'].astype(int).astype(str)

In [10]:
set_of_reliable_ids = set([i['id'] for i in iface.as_table([])]) & set(all_scans.id)

In [11]:
for id in tqdm(set_of_reliable_ids):
    iface.update_sample_record(all_scans[all_scans['id'] == id].iloc[0].to_dict())

  0%|          | 0/530 [00:00<?, ?it/s]

## Generating the dataframes with all measurements available to be saved

In [70]:
from collections import Counter
from itertools import chain

def _transform_db(in_db, measurement_id):
    new_db = in_db[['line', 'id', 'sex', 'age', 'generation', measurement_id]]
    new_db = new_db.rename({o:n for o,n in zip([measurement_id, 'line', 'id', 'sex', 'age', 'generation'], 
                                               ['measurement', 'line_id', 'fish_id', 'fish_sex', 'fish_age', 'fish_generation'])}, axis=1)
    return new_db

def _db_statistics(possible_fields, attempted, whole_db):
    measured = 0
    samples_in_db = []
    for measurement_key in tqdm(possible_fields):
        df = _transform_db(whole_db, measurement_key)
        samples_id_current = list(df.dropna(how='any', subset=['measurement']).fish_id.unique())
    #     print(measurement_key, len(samples_id_current))
        measured += df.dropna(how='any', subset=['measurement']).fish_id.nunique()
        samples_in_db.append(samples_id_current)

    print(measured / len(possible_fields))
    samples_with_that_much_measurements = Counter(Counter(chain.from_iterable(samples_in_db)).values())

    samples_in_db = len(set.union(*[set(i) for i in samples_in_db]))
    print('in db', samples_in_db)

    mc_0 = attempted - samples_in_db
    mc_20 = sum([v for k,v in samples_with_that_much_measurements.items() if k < len(possible_fields)*0.2])
    mc_20_80 = sum([v for k,v in samples_with_that_much_measurements.items() if ((k > len(possible_fields)*0.2) 
                                                                                   and
                                                                                   (k < len(possible_fields)*0.8))])
    mc_80 = sum([v for k,v in samples_with_that_much_measurements.items() if k > len(possible_fields)*0.8])
    mc_100 = sum([v for k,v in samples_with_that_much_measurements.items() if k == len(possible_fields)])

    print('metrics coverage of 0/20/20-40/80/100:', f'{mc_0}/{mc_20}/{mc_20_80}/{mc_80}/{mc_100}')

In [79]:
from glob import glob
import re

In [126]:
# overall
attempted = 340+232
possible_fields = (set([i for rec in iface.as_table([]) for i in rec.keys()]) 
                   - {'line', 'id', 'sex', 'age', 'generation'})
whole_db = pd.DataFrame(iface.as_table([]))
_db_statistics(possible_fields, attempted, whole_db)

  0%|          | 0/108 [00:00<?, ?it/s]

333.6111111111111
in db 538
metrics coverage of 0/20/20-40/80/100: 34/43/298/197/98


In [124]:
qq = pd.read_excel('samplelist_complete.xlsx', sheet_name=4)

In [125]:
qq[qq['S.No'] == 735]

,S.No,Line,Scanned stained,scanned unstained,Phenotyping,Notes,Gen.,Sex,DOB,DOS,Fixation,Dehydration,Staining
335,735,137-4,yes,NaN,NaN,NaN,F12,NaN,2018-08-20,2018-11-05,4% PFA/ 1% GA,Acetone + 0.15% PTA,0.15% PTA- 2days


In [90]:
# 2018_11
addrs_2018_11 = glob('/mnt/LSDF/projects/code-vita/Medaka/2018_11/Medaka_*_*')
ids_2018_11 = [re.findall('Medaka_(\d+)_', i)[0] for i in addrs_2018_11]
lines_2018_11 = [re.findall('Medaka_\d+_(.+)', i)[0] for i in addrs_2018_11]

attempted_2018_11 = len(addrs_2018_11)


fields_eyes = (set([i for rec in iface.as_table([]) for i in rec.keys() if i.startswith('eyes')]) 
                   - {'line', 'id', 'sex', 'age', 'generation'})

whole_db = pd.DataFrame(iface.as_table([]))
db_2018_11 = whole_db[whole_db.id.isin(exp_ids_2018_11)]

_db_statistics(fields_eyes, attempted_2018_11, db_2018_11)

print(db_2018_11[list(fields_eyes)].isna().sum())

  0%|          | 0/29 [00:00<?, ?it/s]

316.37931034482756
in db 324
metrics coverage of 0/20/20-40/80/100: 16/0/0/324/189


In [122]:
glob('/mnt/LSDF/projects/code-vita/Medaka/2018_11/Medaka_735*')

['/mnt/LSDF/projects/code-vita/Medaka/2018_11/Medaka_735_135-1']

In [115]:
pd.Series(Counter(lines_2018_11))

Ho5               58
123-2              6
89-1N              2
58-2              14
80-2              12
131-4_large        3
59-2              11
49-1              16
33-1               8
39-2               6
39-1               9
14-1               5
135-1              7
68-1               7
72-1               7
23-1               9
72-2              12
69-1              13
89-1               9
29-1              11
55-2               6
80-1              14
18-2               8
5-1                9
11-1              10
iCab_unstained     2
50-2              12
32-2_large         5
25-1               3
75-1               4
30-1               8
21-2              14
137-4              3
134-1              4
61-1               7
133-2_large        1
dtype: int64

In [117]:
db_2018_11[db_2018_11.line=='137-4']

,id,eyes.iris.color_average,eyes.iris.color_std,eyes.iris.distance_between_centers,eyes.iris.surface_area,eyes.iris.volume,eyes.lens.color_average,eyes.lens.color_std,eyes.lens.distance_between_centers,eyes.lens.radius_axial,...,heartkidney.right_kidney.eccentricity_equatorial,heartkidney.right_kidney.eccentricity_meridional,heartkidney.right_kidney.surface_area,heartkidney.right_kidney.volume,heartkidney.ventricle.color_average,heartkidney.ventricle.color_std,heartkidney.ventricle.eccentricity_equatorial,heartkidney.ventricle.eccentricity_meridional,heartkidney.ventricle.surface_area,heartkidney.ventricle.volume
60,735,"[141.61239235148153, 128.8980612883052]","[25.20286940542918, 25.301449256950722]",[312.8822186454931],"[31902, 37181]","[41106, 52767]","[234.77236851810872, 233.5200340179994]","[46.329875196719016, 45.179467886728006]",[252.46830291739764],"[[[33.7576897543704, 33.2423102456296], [33.46...",...,[0.3414783535097695],[0.5396632999788827],[64086],[175510],[136.6168307111946],[24.910513195379895],[0.10099621196394097],[0.2295027249177183],[41993],[271979]
113,736,"[127.58605974395448, 131.00834953677227]","[14.71698602453987, 26.14441454332675]",[79.25635076008744],"[1412, 9917]","[703, 8743]","[232.8167590097253, 226.55450523688887]","[43.910204304880814, 55.75660499929508]",[35.834385333468795],"[[[38.79709077517941, 28.202909224820587], [-1...",...,NaN,NaN,[0],[0],[122.66857697743174],[19.684888107956755],[0.09727732223246374],[0.27924586482016833],[64966],[115339]
132,738,"[120.778276481149, 119.62228731194249]","[33.74493646379706, 26.5695166127415]",[254.88670342350173],"[4690, 8395]","[3342, 7511]","[230.53395027137802, 232.3895221215915]","[39.84969293282949, 37.81143563917897]",[208.34127688503767],"[[[26.693446901167164, 27.306553098832836], [2...",...,NaN,NaN,[1560],[1201],[165.07940560690395],[28.785236216599127],[0.13380514492085102],[0.4505104163279234],[34834],[39166]
137,737,"[141.414599717685, 139.79377328366152]","[21.72844087289227, 23.802044034874726]",[339.33619907486985],"[8151, 4616]","[9918, 3758]","[231.74985551811747, 235.0582567381555]","[46.94759777947199, 40.94577789976256]",[244.96432641613262],"[[[33.95957778577417, 33.04042221422583], [34....",...,[0.38082505962427726],[0.6639644906434548],[14111],[27566],[136.91284717789205],[24.03000098202212],[0.16822291162923123],[0.3854737244499167],[40296],[86159]


In [116]:
db_2018_11.groupby('line').id.count()

line
11-1      9
123-2     6
131-4     3
133-2     1
134-1     4
135-1     6
137-4     4
14-1      5
18-2      8
21-2     10
23-1      9
25-1      2
29-1     10
30-1      8
32-2      5
33-1      8
39-1      4
39-2     11
49-1     16
5-1       9
50-2     12
55-2      6
58-2     14
59-2     11
61-1      7
68-1      7
69-1     13
72-1      7
72-2     12
75-1      4
80-1     13
80-2     12
89-1N    11
Ho5      57
Name: id, dtype: int64

In [114]:
db_2018_11.groupby('line').id.count() / pd.Series(Counter(lines_2018_11))

11-1              0.900000
123-2             1.000000
131-4                  NaN
131-4_large            NaN
133-2                  NaN
133-2_large            NaN
134-1             1.000000
135-1             0.857143
137-4             1.333333
14-1              1.000000
18-2              1.000000
21-2              0.714286
23-1              1.000000
25-1              0.666667
29-1              0.909091
30-1              1.000000
32-2                   NaN
32-2_large             NaN
33-1              1.000000
39-1              0.444444
39-2              1.833333
49-1              1.000000
5-1               1.000000
50-2              1.000000
55-2              1.000000
58-2              1.000000
59-2              1.000000
61-1              1.000000
68-1              1.000000
69-1              1.000000
72-1              1.000000
72-2              1.000000
75-1              1.000000
80-1              0.928571
80-2              1.000000
89-1                   NaN
89-1N             5.500000
H

In [ ]:
whole_db[whole_db.id <]

In [ ]:
attempted = 340+232
possible_fields = (set([i for rec in iface.as_table([]) for i in rec.keys()]) 
                   - {'line', 'id', 'sex', 'age', 'generation'})
_db_statistics(possible_fields, attempted)

## Make update to the FTP server

In [10]:
## generating and uploading all datasets

for measurement_key in tqdm(possible_fields):
    df = get_df(iface, measurement_key)
    save_df_to_ftp(df, measurement_key)

In [53]:
list_ftp();

drwxrws---    2 ftp      ftp          4096 Apr 01  2021 backup
-rw-rw----    1 ftp      ftp          7509 Nov 09 14:23 brain.cerebellum.color_average.txt
-rw-rw----    1 ftp      ftp          7506 Nov 09 14:23 brain.cerebellum.color_std.txt
-rw-rw----    1 ftp      ftp          7274 Nov 09 14:23 brain.cerebellum.eccentricity_equatorial.txt
-rw-rw----    1 ftp      ftp          7212 Nov 09 14:23 brain.cerebellum.eccentricity_meridional.txt
-rw-rw----    1 ftp      ftp          5133 Nov 09 14:23 brain.cerebellum.surface_area.txt
-rw-rw----    1 ftp      ftp          5232 Nov 09 14:23 brain.cerebellum.volume.txt
-rw-rw----    1 ftp      ftp          5252 Nov 09 14:23 brain.epyphysis.color_average.txt
-rw-rw----    1 ftp      ftp          5247 Nov 09 14:23 brain.epyphysis.color_std.txt
-rw-rw----    1 ftp      ftp          4565 Nov 09 14:23 brain.epyphysis.surface_area.txt
-rw-rw----    1 ftp      ftp          4568 Nov 09 14:23 brain.epyphysis.volume.txt
-rw-rw----    1 ftp      ftp       